In [1]:
import numpy as np
import pandas as pd

import os, urllib, time
os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlruns/db.sqlite"
import mlflow, git
mlflow_client = mlflow.tracking.MlflowClient()

In [2]:
def leaders(NJ, exp_name, name):
    exp_id = mlflow.get_experiment_by_name(exp_name).experiment_id
    print("# Got exp_id=", exp_id, "for exp_name=", exp_name)
    
    run_name = f"eos.{NJ}J {name}"

    rank_metrics = ['|alpha_tmu|', '|eyelevel_tmu|', '-reward_total']
    print_metrics = ['model_version', 'alpha_tmu', 'alpha_tsigma', 'alpha_tdf', 'eyelevel_tmu', 'eyelevel_tsigma', 'eyelevel_tdf', 'reward_total']

    parent_runs = mlflow_client.search_runs(exp_id, f"tags.mlflow.runName='{run_name}'")
    print("# Got ", len(parent_runs), " parent_runs")

    best_metric = dict()
    best_run = dict()
    
    for parent_run in parent_runs:
        child_runs = mlflow_client.search_runs(exp_id, f"tags.mlflow.parentRunId='{parent_run.info.run_id}'")
        #                                           order_by=["metrics.reward_total DESC"])
        print("# Got ", len(child_runs), " child_runs")
        for child_run in child_runs:
            if 'alpha_tmu' not in child_run.data.metrics:
                continue

            child_run.data.metrics['|alpha_tmu|'] = np.abs(child_run.data.metrics['alpha_tmu'])
            child_run.data.metrics['|eyelevel_tmu|'] = np.abs(child_run.data.metrics['eyelevel_tmu'])
            child_run.data.metrics['-reward_total'] = - child_run.data.metrics['reward_total']

            for metric in rank_metrics:
                if metric not in best_metric or child_run.data.metrics[metric] < best_metric[metric]:
                    best_metric[metric] = child_run.data.metrics[metric]
                    best_run[metric] = child_run
    metric_values = []
    for whats_best, run in best_run.items():
        best_values = [run.data.metrics[metric] for metric in print_metrics]
        metric_values.append([whats_best] + best_values)

    return pd.DataFrame(metric_values, columns=['whats_best'] + print_metrics)

In [3]:
leaders(6, "PYB-6J-3S-1A", "first steps")

# Got exp_id= 3 for exp_name= PYB-6J-3S-1A
# Got  1  parent_runs
# Got  35  child_runs


,whats_best,model_version,alpha_tmu,alpha_tsigma,alpha_tdf,eyelevel_tmu,eyelevel_tsigma,eyelevel_tdf,reward_total
0,|alpha_tmu|,47.0,0.041474,0.002055,0.339474,0.012201,0.006984,0.540385,103.0
1,|eyelevel_tmu|,48.0,0.210225,0.001410,0.393162,0.003601,0.002134,0.393422,0.0
2,-reward_total,54.0,0.074732,0.005339,0.506011,0.043274,0.007814,0.654638,122.0


In [ ]:
#leaders(3, "Train 3J for 3M steps", "training")